# Global Toolbox

In [ ]:
import warnings; warnings.filterwarnings('ignore')

import numpy as np; seed = 356; np.random.seed(seed)
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import sklearn
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, BaggingRegressor, RandomForestRegressor
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from IPython.display import Image
import pandas as pd
from functools import reduce
# !pip install xgboost
from xgboost import XGBClassifier
# !pip install gdown
import gdown
# !pip install pydotplus
import pydotplus 
from tqdm import tqdm

print('The version of my sklearn package is', sklearn.__version__)

# Part 1: Basis Expansion
$f(X) = \sum_{m=1}^{M} \beta_m h_m (X)$
### Piecewise fitting using conditional basis (with two knots _i.e._, $\varepsilon_{1}$ and $\varepsilon_{2}$)


### Initializing coordinates and regions:

In [ ]:
# Axes limits and knots
x_min, x_knot_1, x_knot_2, x_max = -1, 1.5, 4.5, 7

# Points generated from a cosine function with normal noise
x_true = np.linspace(x_min, x_max,50)
y_true = np.cos(x_true)
y_obs  = y_true + np.random.normal(scale=0.5, size=len(x_true))

# Get x-y coordinates per region
x_region_1 = x_true[x_true <= x_knot_1]
x_region_2 = x_true[(x_knot_1 < x_true) & (x_true < x_knot_2)]
x_region_3 = x_true[x_true >= x_knot_2]
y_region_1 = y_true[x_true <= x_knot_1]
y_region_2 = y_true[(x_knot_1 < x_true) & (x_true < x_knot_2)]
y_region_3 = y_true[x_true >= x_knot_2]

### Piecewise constant fitting with the following basis functions:

$h_{1}(X)=I(X<\varepsilon_{1})\\$
$h_{2}(X)=I(\varepsilon_{1}\leq X < \varepsilon_2 )\\$
$h_{3}(X)=I(\varepsilon_{2}\leq X)\\$

In [ ]:
# Plot cosine line and noisy data
plt.figure(figsize=(4, 3))
plt.plot(x_true, y_true, linewidth=3, c='g',  label='True')
plt.scatter(x_true, y_obs, label='True+Noise')

# Plot knots
plt.axvline(x=x_knot_1, c='gray', ls='--', label='Knots')
plt.axvline(x=x_knot_2, c='gray', ls='--')

# Plot piecewise constant fits
plt.axhline(y=y_region_1.mean(), c='r', xmin=0, xmax=0.33, label='Fit')
plt.axhline(y=y_region_2.mean(), c='r', xmin=0.33, xmax=0.66)
plt.axhline(y=y_region_3.mean(), c='r', xmin=0.66, xmax=1)
plt.title('Piecewise Constant')
plt.legend(bbox_to_anchor=(1, 1.03))
plt.xlabel('X')
plt.ylabel('$f(X)$', rotation=0)
plt.show()

### Piecewise linear fitting with the following basis functions:

$h_{1}(X)=I(X<\varepsilon_{1})\\$
$h_{2}(X)=I(\varepsilon_{1}\leq X < \varepsilon_2 )\\$
$h_{3}(X)=I(\varepsilon_{2}\leq X)\\$
$h_{4}(X)=h_1(X)X\\$
$h_{5}(X)=h_2(X)X\\$
$h_{6}(X)=h_3(X)X\\$

In [ ]:
# Calculate OLS coefficients from regression anatomy
beta_region_1 = ((y_region_1 - y_region_1.mean()).dot(x_region_1) / 
                (x_region_1**2).sum())
beta_region_2 = ((y_region_2 - y_region_2.mean()).dot(x_region_2) / 
                (x_region_2**2).sum())
beta_region_3 = ((y_region_3 - y_region_3.mean()).dot(x_region_3) / 
                (x_region_3**2).sum())

# Calculate regression fitted values
y_hat_region_1 = beta_region_1 * x_region_1 + y_region_1.mean()
y_hat_region_2 = beta_region_2 * x_region_2 + y_region_2.mean()
y_hat_region_3 = beta_region_3 * x_region_3 + y_region_3.mean()

# Plot cosine line and noisy data
plt.figure(figsize=(4, 3))
plt.plot(x_true, y_true, linewidth=3, c='g',label='True')
plt.scatter(x_true, y_obs, label='True+Noise')

# Plot knots
plt.axvline(x=x_knot_1, c='gray', ls='--', label='Knots')
plt.axvline(x=x_knot_2, c='gray', ls='--')

# Plot piecewise linear fits
plt.plot(x_region_1, y_hat_region_1, c='r', label='Fit')
plt.plot(x_region_2, y_hat_region_2, c='r')
plt.plot(x_region_3, y_hat_region_3, c='r')
plt.title('Piecewise Linear')
plt.legend(bbox_to_anchor=(1, 1.03))
plt.xlabel('X')
plt.ylabel('$f(X)$', rotation=0)
plt.show()

### Continuous piecewise linear fitting with the following basis functions:

$h_{1}(X)=1\\$
$h_{2}(X)=X\\$
$h_{3}(X)=(X-\varepsilon_1)_+\\$
$h_{4}(X)=(X-\varepsilon_2)_+\\$

In [ ]:
# Construct H
# Continuity is enforced at the knots through incorporating proper constraints into the basis functions
h1 = np.ones_like(x_true)
h2 = np.copy(x_true)
h3 = np.where(x_true < x_knot_1, 0, x_true - x_knot_1)
h4 = np.where(x_true < x_knot_2, 0, x_true - x_knot_2)
H  = np.vstack((h1, h2, h3, h4)).T

# Fit basis expansion via OLS
HH = H.T @ H
beta = np.linalg.solve(HH, H.T @ y_obs)
y_hat = H @ beta

# Plot cosine line and noisy data
plt.figure(figsize=(4, 3))
plt.plot(x_true, y_true, linewidth=3, c='g', label='True')
plt.scatter(x_true, y_obs, label='True+Noise')

# Plot knots
plt.axvline(x=x_knot_1, c='gray', ls='--', label='Knots')
plt.axvline(x=x_knot_2, c='gray', ls='--')

# Plot piecewise linear fits
plt.plot(x_true, y_hat, c='r', label='Fit')
plt.title('Continuous Piecewise Linear')
plt.legend(bbox_to_anchor=(1, 1.03))
plt.xlabel('X')
plt.ylabel('$f(X)$', rotation=0)
plt.show()

### Continuous piecewise polynomial fitting:
Piecewise polynomials, even those continuous at the knots, tend not to be smooth: they rapidly change the slope at the knots. To prevent this and increase their smoothness, it is enough to increase the order of the local polynomial and require the first two derivatives on both sides of the knot to be the same. A function that is continuous and has continuous first and second derivatives is called a cubic spline and can be represented with the following basis functions:

$h_{1}(X)=1\\$
$h_{2}(X)=X\\$
$h_{3}(X)=X^2\\$
$h_{4}(X)=X^3\\$
$h_{5}(X)=(X-\varepsilon_1)_+^3\\$
$h_{6}(X)=(X-\varepsilon_2)_+^3\\$

In [ ]:
# Construct H
h1 = np.ones_like(x_true) # `ones_like` returns an array of ones with the same shape and type as `x_true`
h2 = np.copy(x_true)
h3 = h2 ** 2
h4 = h2 ** 3
h5 = np.where(x_true < x_knot_1, 0, (x_true - x_knot_1) ** 3)
h6 = np.where(x_true < x_knot_2, 0, (x_true - x_knot_2) ** 3)
H  = np.vstack((h1, h2, h3, h4, h5, h6)).T

# Fit basis expansion via OLS
HH = H.T @ H
beta = np.linalg.solve(HH, H.T @ y_true)
y_hat = H @ beta

# Plot simulated data and cubic spline
plt.figure(figsize=(4, 3))
plt.plot(x_true, y_obs, 'o', label='True+Noise')
plt.plot(x_true, y_true, linewidth=3, color='g', label='True')
plt.plot(x_true, y_hat, color='r', label='Fit')
plt.title('Continuous Piecewise Polynomial')

# Plot knots
plt.axvline(x=x_knot_1, c='gray', ls='--', label='Knots')
plt.axvline(x=x_knot_2, c='gray', ls='--')
plt.legend(bbox_to_anchor=(1, 1.03))             
plt.xlabel('X')
plt.ylabel('$f(X)$', rotation=0)
plt.show()

# Part 2: Decision Trees

Review Scikit-Learn [Decision Trees](https://scikit-learn.org/stable/modules/tree.html).

In [ ]:
# Make some fake data
def make_regression_data(n=25):
    x = np.random.uniform(size = (n,1))
    e = np.random.normal(0, 0.3, size = x.shape) # some noise
    y = np.sin(2*np.pi*x) + e
    return (x, y.ravel())

x,y = make_regression_data()
fig, ax = plt.subplots(dpi = 120)
plt.scatter(x,y)
plt.show()

In [ ]:
# Fit a regression tree to this data
# With varying max depth
# To see how this tree grows

depths = np.arange(1,7)
fig, ax = plt.subplots(dpi = 120, nrows = 2, ncols = 3, figsize = (10,6))
plt.subplots_adjust(hspace=0.25)
ax = ax.ravel() #flattens the ax variable

newx = np.linspace(0,1,101).reshape(-1,1)

for d,a in zip(depths, ax):
    reg = DecisionTreeRegressor(max_depth=d)
    reg.fit(x,y)
    
    ypred = reg.predict(newx)
    a.scatter(x,y)
    a.plot(newx, ypred, color = 'red')
    a.set_title(f'max_depth = {d}')

In [ ]:
# Create DOT data
dot_data = export_graphviz(reg, out_file=None, rounded = True, max_depth=3, feature_names=['x'])

# Draw graph
graph = pydotplus.graph_from_dot_data(dot_data)  

# Show graph
Image(graph.create_png())

In [ ]:
# Let's compare variance of predictions between a tree and a bagged estimator

# Create a tree model and a bagging from these trees
newx = np.linspace(0,1,101).reshape(-1,1)
tree = DecisionTreeRegressor(max_depth = 5)
bag1 = BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth = 5), n_estimators=100, n_jobs = -1)

models = [tree, bag1]
model_names = ['Tree', "Bagged"]

# Create two plots, one for the tree, one for the bagging
fig, ax = plt.subplots(dpi = 120, nrows = 1, ncols = len(models), figsize = (10,3), sharey = True)
ax = ax.ravel() #flattens the ax variable

# Generate 500 runs for each model and calculate bias and variance. This takes a while!
nsim = 50

for axis, model, name in zip(ax, models, model_names):
    
    #Store the predictions somewhere
    predictions = np.zeros((nsim, newx.shape[0]))
    
    for i in range(nsim):
        x,y = make_regression_data()
        model.fit(x,y)
        ypred = model.predict(newx)
        predictions[i] = ypred
        
    #Plot the variance
    axis.plot(newx, predictions.var(axis = 0), label = 'Variance')
    
    #Plot the bias
    bias = (predictions - np.sin(2*np.pi*newx.T)).mean(axis = 0)**2
    axis.plot(newx, bias, color = 'red', label = 'Squared Bias')
    axis.set_title(name)
    axis.legend()

The following cell is just the expanded version of the previous cell to make it easier to follow, otherwise they do the same thing.

In [ ]:
# Let's compare variance of predictions between a tree and a bagged estimator
newx = np.linspace(0,1,101).reshape(-1,1)

# Generate `nsim` runs for each model and calculate bias and variance. This takes a while!
nsim = 50

# Create two plots, one for the tree, one for the bagging
fig, ax = plt.subplots(dpi=120, nrows=1, ncols=len(models), figsize=(10,3), sharey=True)


################################################### 1st figure
# Create a simple tree model
tree = DecisionTreeRegressor(max_depth=5)
# Initialize an array for predictions
predictions = np.zeros((nsim, newx.shape[0]))

for i in range(nsim):
    x, y = make_regression_data() # every time get new data from the same data generating process
    tree.fit(x,y)
    ypred = tree.predict(newx) # here ypred is coming directly from the terminal 
                               # nodes of the single tree fitted to x and y
    predictions[i] = ypred
    
#Plot the variance
ax[0].plot(newx, predictions.var(axis = 0), label = 'Variance')

#Plot the bias
bias = (predictions-np.sin(2*np.pi*newx.T)).mean(axis = 0)**2
ax[0].plot(newx, bias, color = 'red', label = 'Squared Bias')
ax[0].set_title('Tree')


################################################### 2nd figure
# Create a bagging model
bag = BaggingRegressor(DecisionTreeRegressor(max_depth=5), n_estimators=100, n_jobs=-1)
# Initialize an array for predictions
predictions = np.zeros((nsim, newx.shape[0]))

for i in range(nsim):
    x, y = make_regression_data() # every time get new data from the same data generating process
    bag.fit(x,y)
    ypred = bag.predict(newx) # here ypred is coming from averaging individual predictions of
                              # random subsets of the original dataset (one tree per random subset)
    predictions[i] = ypred
    
#Plot the variance
ax[1].plot(newx, predictions.var(axis = 0), label = 'Variance')

#Plot the bias
bias = (predictions-np.sin(2*np.pi*newx.T)).mean(axis=0)**2
ax[1].plot(newx, bias, color = 'red', label = 'Squared Bias')
ax[1].set_title('Bagged')
ax[1].legend()

plt.show()

# Part 3: Random Forest

Now we will train a random forest. It is included in the ```sklearn.ensemble``` subpackage, function [```RandomForestClassifier```](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), so it is straightforward to use. It comes with many parameters, but in general there is a philosophy to follow:

- In a Random Forest we want each tree to be large, and to learn as much as possible from its subset of data. We don't care too much if each tree is overadjusted, as we can always increase the number of trees to take care of this.

- This said, a good idea is to limit the minimum number of samples per leaf when we have few cases (this is not usually a problem in large trees.)

- We might want to limit the minimum impurity decrease to stop growing a tree if not much is happening.

- There is also a class weight to include. It does include one automatically if we use the option ```balanced```.

Let's train one and check the options.

In [ ]:
#Define the classifier
bankloan_rf = 

# These are parameters to limit the growth of the tree. If you need to constrain the growth of the tree, pick one of them.
# max_depth
# min_samples_split
# min_samples_leaf  
# min_weight_fraction_leaf
# min_impurity_decrease  # this is more agnostic to the sample size. If you really need to constrain growth, this is preferred 

Now we are ready to train. I have created a credit risk dataset that can be used to predict the probability of not paying back a loan (a credit score). We will also split the data into a train and test set.

In [ ]:
# Read the file
bankloan_data = 

# Drop a categorical variable
bankloan_data.

The data has the following variables:

- Customer: ID, or unique label, of the borrower (NOT predictive).
-    Age: Age of the borrower in years.
-    Employ: Years at current job.
-    Address: Years at current address.
-    Income: Income in ‘000s USD.
-    Leverage: Debt/Income Ratio.
-    CredDebt: Credit card standing debt.
-    OthDebt: Other debt in ‘000s USD.
-    MonthlyLoad: Monthly percentage from salary used to repay debts.
-    Default: 1 If default has occurred, 0 if not (Target variable).


In [ ]:
print(bankloan_data.shape)
bankloan_data.head()

In [ ]:
bankloan_data.

Now we split the data into train and test.

In [ ]:
# Split into train and test, fixing seed.
bankloan_train_noWoE, bankloan_test_noWoE = 

Now we train!

In [ ]:
# Train the RF
bankloan_rf.

We can see it used two jobs (two processors are available to me in this computer). It converges very quickly. Let's check how it did, this time we will print a nicer confusion matrix using seaborn, and will plot the ROC curve of the model. 

In [ ]:
# Apply the model to the test set.
rf_pred_class_test = bankloan_rf.
rf_probs_test = bankloan_rf.

In [ ]:
# Calculate confusion matrix
confusion_matrix_rf = 

# Turn matrix to percentages
confusion_matrix_rf = 

# Turn to dataframe
df_cm = pd.DataFrame(confusion_matrix_rf, index=['Good', 'Bad'], columns=['Good', 'Bad'])

# Parameters of the image
figsize = (10,7)
fontsize=14

# Create image
fig = plt.figure(figsize=figsize)
heatmap = sns.heatmap(df_cm, annot=True, fmt='.2f')

# Make it nicer
heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0,  ha='right', fontsize=fontsize)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)

# Add labels
plt.ylabel('True label')
plt.xlabel('Predicted label')

# Plot!
plt.show()

[Out-of-Bag (OOB) error](https://scikit-learn.org/stable/auto_examples/ensemble/plot_ensemble_oob.html)

The `RandomForestClassifier` is trained using bootstrap aggregation, where each new tree is fit from a bootstrap sample of the training observations $z_i=(x_i,y_i)$. The out-of-bag (OOB) error is the average error for each $z_i$ calculated using predictions from the trees that do not contain $z_i$ in their respective bootstrap sample. This allows the `RandomForestClassifier` to be fit and validated whilst being trained.

In [ ]:
bankloan_rf
# can think of this as validation score

Return the mean accuracy on the given test data and labels:

In [ ]:
bankloan_rf.

Looks a bit unbalanced, but otherwise ok. It's harder to predict the  defaulters. Now let's see the ROC curve.

In [ ]:
# Calculate the ROC curve points
fpr, tpr, thresholds = 

# Save the AUC in a variable to display it. Round it first
auc = 

# Create and show the plot
plt.plot(fpr,tpr,label="Bankloan RF, auc="+str(auc))
plt.legend(loc=4)
plt.show()

Now, let's print the variable importance. The importance is calculated by averaging the accuracy of trees when the variables is included the tree, and comparing it to when it's NOT included the tree.

[Feature Importances with a Forest of Trees](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html)
also [this](https://towardsdatascience.com/the-mathematics-of-decision-trees-random-forest-and-feature-importance-in-scikit-learn-and-spark-f2861df67e3).

In [ ]:
importances = 
print(bankloan_rf.feature_names_in_)
print(importances)

In [ ]:
# Plot variable importance
indices = np.argsort(importances)[::-1] 

f, ax = plt.subplots(figsize=(3, 8))

plt.title("Variable Importance - Random Forest")
sns.set_color_codes("pastel")

sns.barplot(y=[bankloan_rf.feature_names_in_[i] for i in indices], x=importances[indices], label="Total", color="b")
# or
# sns.barplot(y=[bankloan_train_noWoE.iloc[:, :-1].columns[i] for i in indices], x=importances[indices], label="Total", color="b")

ax.set(ylabel="Attribute", xlabel="Attribute Importance (Entropy)")
sns.despine(left=True, bottom=True)

Also take a look at [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html).

# Part 4: [XGBoosting](https://xgboost.readthedocs.io/en/stable/)

XGBoost is an optimized distributed gradient boosting library designed to be highly efficient, flexible and portable. It implements machine learning algorithms under the Gradient Boosting framework and is an alternative to Random Forest. Now we want to create a series of small trees, which will be poorer in individual performance, but together they will be stronger. Training an XGBoost model is harder, because we need to control the model so it creates small trees, but it performs better in small data, something Random Forests do not necessarily accomplish.

[XGBClassifier Scikit-Learn API](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBClassifier)

[XGBRegressor Scikit-Learn API](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBRegressor)

While scikit-learn does have its own implementation of XGB ([```sklearn.ensemble```](https://scikit-learn.org/stable/modules/ensemble.html)), there are a couple of very strong packages out there that implement the algorithm. ```xgboost``` and ```lightgbm``` are two of the best known ones. We will use ```xgboost``` for this lab, available pretty much for every language out there.

The first step is to define a classifier that we will use.

In [ ]:
#Define the classifier.

# Specify the learning task and the corresponding learning objective. The objective options are below:
# https://xgboost.readthedocs.io/en/stable/parameter.html#learning-task-parameters

XGB_Bankloan = XGBClassifier(

In [ ]:
bankloan_data.Default.value_counts()

# # we are interested in the number of default cases --> positive class
# print(bankloan_data.Default.sum())

# # number of non-default cases --> negative class
# print(bankloan_data.shape[0]-bankloan_data.Default.sum())

This classifier can be used to tune the parameters of the model. We will use sklearn's  [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) for this. It requires a dictionary of the parameters to look for. We will tune the number of trees (XGB overfits relatively easily, always tune this), the depth, and the learning rate.

In [ ]:
# Define the parameters. Play with this grid!
param_grid = 
param_grid

Recap: [Stratified Sampling](https://en.wikipedia.org/wiki/Stratified_sampling)

We will create a validation set for the sample.

In [ ]:
# Always a good idea to tune on a reduce sample of the train set, as we will call many functions.
val_train = 


# Crossvalidation object
cv_object = 

Now we can do a [grid search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) over the parameter space. We will use the AUC (as this is a binary classification problem).

In [ ]:
# Define grid search object.
GridXGB = 

In [ ]:
# Train grid search.
GridXGB.

Now we can output the optimal parameters.

In [ ]:
# Show best params
GridXGB.

It is telling us to use 10% learning rate with a max_depth of 2 and 50 trees. If any parameter appears to be at the upper limit, you would need to run it again with a higher limit.

We can get the details of the runs accessing cv_results_

In [ ]:
GridXGB.

Now we can fit the final model!

In [ ]:
# Let's recreate XGB with the best parameters:
XGB_Bankloan = XGBClassifier(max_depth=          # Depth of each tree
                             learning_rate=      # How much to shrink error in each subsequent training. Trade-off with no. estimators.
                             n_estimators=       # How many trees to use, the more the better, but decrease learning rate if many used.
                             verbosity=1,                  # If to show more errors or not.
                             objective='binary:logistic',  # Type of target variable.
                             booster='gbtree',             # What to boost. Trees in this case.
                             n_jobs=4,                     # Parallel jobs to run. Set your processor number.
                             gamma=0.001,                  # Minimum loss reduction required to make a further partition on a leaf node of the tree. (Controls growth!)
                             subsample=0.632,              # Subsample ratio. Can set lower
                             colsample_bytree=1,           # Subsample ratio of columns when constructing each tree.
                             colsample_bylevel=1,          # Subsample ratio of columns when constructing each level. 0.33 is similar to random forest.
                             colsample_bynode=1,           # Subsample ratio of columns when constructing each split.
                             reg_alpha=1,                  # Regularizer for first fit. alpha = 1, lambda = 0 is LASSO.
                             reg_lambda=0,                 # Regularizer for first fit.
                             scale_pos_weight=1,           # Balancing of positive and negative weights.
                             base_score=0.5,               # Global bias. Set to average of the target rate.
                             random_state=seed
                            )

In [ ]:
# Train over all training data.
XGB_Bankloan.

Now we can evaluate our model. First we calculate the variable importance.

In [ ]:
# Plot variable importance
importances = 
indices = 

f, ax = plt.subplots(figsize=(3, 8))
plt.title("Variable Importance - XGBoosting")
sns.set_color_codes("pastel")
sns.barplot(y=[bankloan_train_noWoE.iloc[:, :-1].columns[i] for i in indices], x=importances[indices], 
            label="Total", color="b")
ax.set(ylabel="Variable",
       xlabel="Variable Importance (Gini)")
sns.despine(left=True, bottom=True)

What do you see here? Does it make sense to you?

When the correlation between the variables are high, XGBoost will pick one feature and may use it while breaking down the tree and it will ignore some/all the other remaining correlated features (because we will not be able to learn different aspects of the model by using these correlated feature because it is already highly correlated with the chosen feature). But in random forest , the tree is not built from specific features, rather there is random selection of features, and then the model in whole learn different correlations of different features. So you can see the procedure of two methods are different so you can expect to see different feature importance reports. Also, one needs to check default arguments of each method for calculation of FI. For example, XGBoost's default measure is average gain whereas it is total gain in sklearn. 

Let's finish by plotting the ROC curve. How does it compare to Random Forest? Why do you think this is?

In general, boosting methods work better for small datasets with fewer features.  

In [ ]:
# Calculate probability
probTest = 
probTest = 

# Calculate the ROC curve points
fpr, tpr, thresholds = 

# Save the AUC in a variable to display it. Round it first
auc = 

# Create and show the plot
plt.plot(fpr,tpr,label="AUC - XGBoosting = " + str(auc))
plt.legend(loc=4)
plt.show()